# ShopAssist AI




#### Project Background

In today's digital age, online shopping has become the go-to option for many consumers. However, the overwhelming number of choices and the lack of personalized assistance can make the shopping experience daunting. To address this, we have developed **ShopAssist AI, a chatbot that combines the power of large language models and rule-based functions to ensure accurate and reliable information delivery**.


#### Problem Statement

*Given a dataset containing information about laptops (product names, specifications, descriptions, etc.), build a chatbot that parses the dataset and provides accurate laptop recommendations based on user requirements*.






In [ ]:
import pandas as pd
df = pd.read_csv('laptop_data2.csv')
df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"2,00,000",The ASUS ZenBook Pro is a high-end laptop that...


#### Approach:

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play, extracting top 3 laptops that best matches the user's needs.
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect laptop solution.

## System Design

#### Dataset

We have a dataset `laptop.csv` where  each row describes the features of a single laptop and also has a small description at the end. The chatbot that we build will leverage LLMs to parse this `Description` column and provide recommendations

#### Workings of the Chatbot

**Note1** - Don't run this when you go through this section for the first time. Run this only after running through the rest of the code. When going through this for the first time, please only go through the output, to understand the workings of the chatbot

**Note2** - The conversation shown here might be different from what you see in the video. However, the overall structure of the conversation to understand the system design remains the same.

In [ ]:
dialogue_mgmt_system()

Assistant: "Hello! I am an intelligent laptop gadget expert. I can help you find the best laptop based on your requirements. Please let me know what you will be using the laptop for and any specific needs or preferences you have."

I am Kaustubh and I work as a management consultant.

Great! As a management consultant, you likely require a laptop that can handle multitasking and data analysis tasks efficiently. May I know what kind of software or tools do you primarily use for your work? Understanding the specific software will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now.



No

I primarily work on Excel and powerpoint

Thank you for providing that information. Working with Excel and PowerPoint involves handling large datasets and creating presentations, which requires good processing speed and multitasking capability. Do you often work with complex formulas and macros in Excel? Understanding the complexity of your tasks will help

As you could see in the output above, the chatbot should ask a series of questions to
- Determine the user's requirments. For simplicity, we have used 6 features to encapsulate the user's needs. The 6 features are as follows:
    - GPU intensity
    - Display quality
    - Portability
    - Multitasking
    - Processing speed
    - Budget

- Confirm if the user's requirements have been correctly captured at the end.

After that the chatbot lists down the top 3 products that are the most relevant, and engages in further conversation to help the user find the best one.


#### Building the Chatbot



![Chatbot_sys_design.png](https://drive.google.com/uc?id=1j-mw_dNcbxGcelQ0PmkDB0nKOpauU1wX)

As shown in the image, the chatbot contains the following layers:
- Intent Clarity Layer
- Intent Confirmation Layer
- Product Mapping Layer
- Product Information Extraction Layer
- Product Recommendation Layer

##### Major functions behind the Chatbot

Let's now look at a brief overview of the major functions that form the chatbot. We'll take a deep dive later



- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_model_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget
- `dictionary_present()`: This function checks if the final understanding of user's profile is returned by the chatbot as a python dictionary or not. If there is a dictionary, it extracts the information as a Python dictionary.
- `compare_laptops_with_user()`: This function compares the user's profile with the different laptops and come back with the top 3 recommendations.
- `initialize_conv_reco()`: Initializes the recommendations conversation

## Implementation

### Import the libraries

Let's start by importing the libraries that we'll require for this project. Following are the ones:
- openai
- pandas
- os, json, ast

The api key is in a text file api_key.txt.

In [ ]:
# !pip install openai

In [ ]:
import os,json,ast
import openai
import pandas as pd

In [ ]:
# read the API key
openai.api_key = open("api_key.txt", "r").read().strip()

### Implementing Intent Clarity and Intent Confirmation Layers

Let's start with the first part of the implementation - building the `intent clarity` and `intent confirmation` layers. As mentioned earlier, this layer helps in identifying the user requirements and passing it on to the product matching layer. Here are the functions that we would be using for building these layers:

- `initialize_conversation()`: This initializes the variable conversation with the system message. Using prompt engineering and chain of thought reasoning, the function will enable the chatbot to keep asking questions until the user requirements have been captured in a dictionary. It also includes Few Shot Prompting(sample conversation between the user and assistant) to align the model about user and assistant responses at each step.


In [ ]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"
    example_user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'low','Multitasking': 'high','Processing speed': 'high','Budget': '150000'}

    system_message = f"""

    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    The value for 'budget' should be a numerical value extracted from the user's response.
    The values currently in the dictionary are only representative values.

    {delimiter}Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised.
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys. The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    {delimiter} Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    You are trying to fill the values of all the keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    Answer "Yes" or "No" to indicate if you understand the requirements and have updated the values for the relevant keys. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n{delimiter}

    {delimiter}Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys. Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    Answer "Yes" or "No" to indicate if you understood all the values for the keys and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.{delimiter}

    {delimiter}Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions. {delimiter}

    Follow the above chain of thoughts and only output the final updated python dictionary. \n


    {delimiter} Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_req}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

Let's see what does `initialize_conversation()` does. We have added a prefix `debug_` to each of the variables so that we can play around with the inputs and outputs and it doesn't disturb the main function.

In [ ]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The values for all keys, except \'budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stated by user. \n    The value for \'budget\' should be a numerical value extracted from the user\'s 

Let's now look at the next function.
- `get_chat_model_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant

In [ ]:
# recall that messages is a list of dicts
# [{"role": "system", "content": system_message},
# {"role": "user", "content": user_input},
# {"role": "assistant", "content": assistant_message}]

In [ ]:
def get_chat_model_completions(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 3000
    )
    return response.choices[0].message["content"]

Let's pass the initialized conversation `debug_conversation` and see what is the assistant's response.

In [ ]:
debug_introduction = get_chat_model_completions(debug_conversation)
print(debug_introduction)

Hello! I am an intelligent laptop gadget expert and I am here to help you find the best laptop for your needs. Please let me know what you will be using the laptop for and any specific requirements you have.


Let's play around a bit and add the following user's input `debug_user_input` to the conversation `debug_conversation` and see what the assistant responds with.

In [ ]:
debug_user_input = "Hi, I am Anand. I need a laptop for coding."

In [ ]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Great! As a coder, you likely require a laptop that can handle multitasking and has a good processing speed. May I know what kind of coding work do you primarily focus on? Are you more involved in web development, data analysis, or software development? Understanding the specific type of coding work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now.


Typically, whenever the chatbot is interacting with the user, all the conversations should be moderated to identify any inappropriate content. Let's look at the function that can help with it.

- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, you can add a break statement to end the conversation.

In [ ]:
def moderation_check(user_input):
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]
    if moderation_output["flagged"] == True:
        return "Flagged"
    else:
        return "Not Flagged"

Let's test moderation on the `debug_user_input`

In [ ]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Not Flagged


Let's now test moderation on some other text.

In [ ]:
print(moderation_check("I want to kill Ravan."))
print(moderation_check("I need a laptop to hack NASA."))

Flagged
Not Flagged


So, this moderation api may not be perfect but if you ask this to the ChatGPT or it's API (GPT 3.5), it'll not help you with such requests. Remember, moderation should also be applied on the GPT 3.5's output.

Let's now check moderation on the assistant's response `debug_response_assistant`.

In [ ]:
moderation_check(debug_response_assistant)

'Not Flagged'

As mentioned earlier, you need to understand the user's profile, which essentially means that all the features: GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget are captured or not. Let's look at the function that helps us verify that.

- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not
   - GPU intensity
   - Display quality
   - Portability
   - Multitasking
   - Processing speed
   - Budget

In [ ]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt = f"""
    You are a senior evaluator who has an eye for detail.
    You are provided an input. You need to evaluate if the input has the following keys: 'GPU intensity','Display quality','Portability','Multitasking',' Processing speed','Budget'
    Next you need to evaluate if the keys have the the values filled correctly.
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance as stated by user. The value for the key 'budget' needs to contain a number with currency.
    Output a string 'Yes' if the input contains the dictionary with the values correctly filled for all keys.
    Otherwise out the string 'No'.

    Here is the input: {response_assistant}
    Only output a one-word string - Yes/No.
    """


    confirmation = openai.completions.create(
                                    model="text-davinci-003",
                                    prompt = prompt,
                                    temperature=0)


    return confirmation["choices"][0]["text"]

In [ ]:
# Here are some sample input output pairs for better understanding:
#     {delimiter}
#     input: "{{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'low'}}"
#     output: No

#     input: "{{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': '', 'Budget': '90000'}}"
#     output: No

#     input: "Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'low','Processing speed': 'high','Budget': '200000'"
#     output: Yes

#     input: "Here is your recommendation {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'low', 'Budget': '90000'}}"
#     output: Yes

#     input: "Here is your recommendation - 'GPU intensity': 'high' - 'Display quality': 'low' - 'Portability': 'low'  - 'Multitasking': 'high' - 'Processing speed': 'high' - 'Budget': '90000' "
#     output: Yes

#     input: "You can look at this - GPU intensity: high - Display quality: low - Portability: low  - Multitasking: high - Processing speed: high - Budget: 90000"
#     output: Yes

#     input: "{{GPU intensity: low, Display quality: high, Portability: low, Multitasking:high,Processing speed: Low, Budget: 70000}}"
#     output: No

#     {delimiter}

Let's apply the function to the assistant's reponse and see if it has captured the user profile.

In [ ]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
print(debug_confirmation)


No


Now, you can keep adding user and assistant responses to debug_conversation and get to a point where intent_confirmation_layer() gives yes as a response. Let's see if the following response by the assistant passes the intent_confirmation_layer() test.

In [ ]:
#Let's add the above assistant response to the debug_conversation.
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})

Let's say that after a series of conversations you get the following response from the assistant.

In [ ]:
debug_response_assistant_n = f"""

Based on the information you have provided, I have updated the values in the dictionary as follows:

{{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'low', 'Budget': '90000'}}

Please note that the values for 'Display quality' and 'Portability' are set to 'low' since they are not a priority for your coding tasks.

Is there anything else I can assist you with?
"""
#Note that you are using double curly braces

Do you think it'll pass the `intent_confirmation_layer()` test? Let's try it out.

In [ ]:
intent_confirmation_layer(debug_response_assistant_n)

'\nYes'

Let's now look at the working of `dictionary_present()`.

- `dictionary_present()`: This function checks if the final understanding of user's profile is returned by the chatbot is a Python dictionary or not. This is important as it'll be used later on for finding the right laptops using dictionary matching.

In [ ]:
def dictionary_present(response):
    delimiter = "####"
    user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'}
    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.

            Here are some sample input output pairs for better understanding:
            {delimiter}
            input: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input: {{'GPU intensity':     'low', 'Display quality':     'high', 'Portability':    'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90,000'}}
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90000'}}

            input: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'low','Processing speed': 'high','Budget': '200000 INR'
            output: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'low','Budget': '200000'}}
            {delimiter}

            Here is the input {response}

            """
    response = openai.completions.create(
        model="text-davinci-003",
        prompt=prompt,
        max_tokens = 2000
        # temperature=0.3,
        # top_p=0.4
    )
    return response["choices"][0]["text"]

Let's start by passing the `debug_response_assistant`.

In [ ]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)


Output: {'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'low', 'Budget': '90000'}


What if you pass something like this where it is not in the form of a dictionary? Or some key or some values are missing? Let's see.

In [ ]:
debug_response_assistant_n = f"""Thank you for providing your budget. Based on your budget of 50,000 INR, I will consider this while recommending suitable laptop options for you.
Here is the final recommendation for your laptop:

- GPU intensity: high
- Display quality: high
- Portability: low
- Multitasking: high
- Processing speed: medium
- Budget: 80,000 INR

Please note that these specifications are based on your requirements for surfing and a decent display within your budget. Let me know if there's anything else I can assist you with!"""

In [ ]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)


Output: {'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '80000'}


Let's quickly take a look at the code that we have run until now.

In [ ]:
debug_conversation = initialize_conversation()
debug_introduction = get_chat_model_completions(conversation)
debug_user_input = "Hi, I am Anand. I need a laptop for coding."
debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
.
.
.
response_dict_n = dictionary_present(debug_response_assistant_n)

So, now that you have the user profile stored in `response_dict_n`. We'll use this to generate recommendations. Before that, we need to create a similar profile for every laptop. Let's see how we do it.

### Implementing the Product Mapping and Information Extraction Layers

In this section, we take in the output of the previous layers, i.e. the user requirements, which is in the format of a Python dictionary, and extract the top 3 laptop recommendations based on that. Here are the functions that we will use to help us implement the information extraction and product matching layers


- `product_map_layer()`: This function is responsible for extracting key features and criteria from laptop descriptions. Here's a breakdown of how it works:
    - Uses a prompt that assign it the role of a Laptop Specifications Classifier, whose objective is to extract key features and classify them based on laptop descriptions.
    - Provide step-by-step instructions for extracting laptop features from description.
    - Assign specific rules for each feature (e.g., GPU Intensity, Display Quality, Portability, Multitasking, Processing Speed) and associate them with the appropriate classification value (Low, Medium, or High).
    - Includes Few Shot Prompting(sample conversation between the user and assistant) to demonstrate the expected result of the feature extraction and classification process.

In [ ]:
def product_map_layer(laptop_description):

  delimiter = "#####"
  lap_spec = {
      "GPU intensity":"(Type of the Graphics Processor)",
      "Display quality":"(Display Type, Screen Resolution, Display Size)",
      "Portability":"(Laptop Weight)",
      "Multitasking":"(RAM Size)",
      "Processing speed":"(CPU Type, Core, Clock Speed)"
  }

  values = {'low','medium','high'}

  prompt=f"""
  You are a Laptop Specifications Classifier whose job is to extract the key features of laptops as per their requirements.
  To analyze each laptop, perform the following steps:
  Step 1: Extract the laptop's primary features from the description {laptop_description}
  Step 2: Store the extracted features in {lap_spec} \
  Step 3: Classify each of the items in {lap_spec}into {values} based on the following rules: \
  {delimiter}
  GPU Intensity: low: < for Integrated graphics or entry-level dedicated graphics like Intel UHD > , \n
  medium: < if Mid-range dedicated graphics like M1, AMD Radeon, Intel Iris > , \n
  high: < High-end dedicated graphics like Nvidia > , \n

  Display Quality: low: < if resolution below Full HD (e.g., 1366x768). > , \n
  medium: < if Full HD resolution (1920x1080) or higher. > , \n
  high: < if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. > \n

  Portability: low: < if laptop weight is less than 1.51 kg > , \n
  medium: < if laptop weight is between 1.51 kg and 2.51 kg> , \n
  high: <if laptop weight is greater than 2.51 kg> \n

  Multitasking: low: < If RAM size is 8GB, 12GB > , \n
  medium: < if RAM size is 16GB > , \n
  high: < if RAM size is 32GB, 64GB> \n

  Processing Speed: low: < if entry-level processors like Intel Core i3, AMD Ryzen 3> , \n
  medium: < if Mid-range processors like Intel Core i5, AMD Ryzen 5 > , \n
  high: < if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher > \n
  {delimiter}

  {delimiter}
  Here are some input output pairs for few-shot learning:
  input1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
  output1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

  input2: "The Lenovo ThinkPad X1 Carbon is a sleek and lightweight laptop designed for professionals on the go. It is equipped with an Intel Core i7 processor running at 2.6 GHz, providing strong processing capabilities for multitasking and productivity. With 16GB of RAM and an SSD, it offers fast and efficient performance along with ample storage capacity. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It comes with Intel UHD integrated graphics for decent graphical performance. Weighing just 1.13 kg, it is extremely lightweight and highly portable. The laptop features an IR camera for face unlock, providing convenient and secure login options. With a three-year warranty and an impressive battery life of up to 12 hours, the Lenovo ThinkPad X1 Carbon ensures reliability and long-lasting productivity. Priced at 130,000, it offers top-notch performance and portability for professionals."
  output2: {{'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'high', 'Multitasking':'high', 'Processing speed':'high'}}

  input3: "The Apple MacBook Pro is a high-end laptop that combines top-tier performance with a stunning display. It is equipped with an Intel Core i9 processor running at 2.9 GHz, providing exceptional processing power for demanding tasks and content creation. With 32GB of RAM and an SSD, it offers seamless multitasking and fast storage access for large projects. The laptop features a 16" Retina display with a resolution of 3072x1920, delivering breathtaking visuals and precise color reproduction. It comes with an AMD Radeon graphics card, ensuring smooth graphics performance for professional applications. Weighing 2.02 kg, it is relatively lightweight for its size. The laptop features a True Tone display, adjusting the color temperature to match the ambient lighting for a more natural viewing experience. With a three-year warranty and a battery life of up to 10 hours, the Apple MacBook Pro offers reliability and endurance for professionals. Priced at 280,000, it caters to users who require uncompromising performance and a superior display for their demanding workloads."
  output3: {{'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'medium','Multitasking': 'high', 'Processing speed': 'high'}}
  {delimiter}

  Follow the above instructions step-by-step and output the dictionary {lap_spec} for the following laptop {laptop_description}.
  """

#see that we are using the Completion endpoint and not the Chatcompletion endpoint

  response = openai.completions.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens = 2000,
    # temperature=0.3,
    # top_p=0.4
    )
  return response["choices"][0]["text"]

In [ ]:
##Run this code once to extract product info in the form of a dictionary
laptop_df= pd.read_csv('laptop_data2.csv')

## Create a new column "laptop_feature" that contains the dictionary of the product features
laptop_df['laptop_feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))

In [ ]:
laptop_df.to_csv("updated_laptop.csv",index=False,header = True)

- `extract_dictionary_from_string()`: This function takes in the output of the previous layer and extracts the user requirements dictionary

In [ ]:
import ast
import re

def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

In [ ]:
string = " \n \t           Output: {{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}ad"
extracted_dict = extract_dictionary_from_string(string)
print(extracted_dict)

{'gpu intensity': 'high', 'display quality': 'high', 'portability': 'low', 'multitasking': 'high', 'processing speed': 'medium', 'budget': '50000'}


- `compare_laptops_with_user()`: This function compares the user's profile with the different laptops and come back with the top  recommendations. It will perform the following steps:
    - It will take the user requirements dictionary as input
    - Filter the laptops based on their price, keeping only the ones within the user's budget.
    - Calculate a score for each laptop based on how well it matches the user's requirements.
    - Sort the laptops based on their scores in descending order.
    - Return the top 3 laptops as a JSON-formatted string.

In [ ]:
def compare_laptops_with_user(user_req_string):
    laptop_df= pd.read_csv('updated_laptop.csv')
    user_requirements = extract_dictionary_from_string(user_req_string)
    budget = int(user_requirements.get('budget', '0').replace(',', '').split()[0])
    #This line retrieves the value associated with the key 'budget' from the user_requirements dictionary.
    #If the key is not found, the default value '0' is used.
    #The value is then processed to remove commas, split it into a list of strings, and take the first element of the list.
    #Finally, the resulting value is converted to an integer and assigned to the variable budget.


    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()
    #These lines create a copy of the laptop_df DataFrame and assign it to filtered_laptops.
    #They then modify the 'Price' column in filtered_laptops by removing commas and converting the values to integers.
    #Finally, they filter filtered_laptops to include only rows where the 'Price' is less than or equal to the budget.

    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    # Create 'Score' column in the DataFrame and initialize to 0
    filtered_laptops['Score'] = 0
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['laptop_feature']
        laptop_values = extract_dictionary_from_string(user_product_match_str)
        score = 0

        for key, user_value in user_requirements.items():
            if key.lower() == 'budget':
                continue  # Skip budget comparison
            laptop_value = laptop_values.get(key, None)
            laptop_mapping = mappings.get(laptop_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if laptop_mapping >= user_mapping:
                ### If the laptop value is greater than or equal to the user value the score is incremented by 1
                score += 1

        filtered_laptops.loc[index, 'Score'] = score

    # Sort the laptops by score in descending order and return the top 5 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

    return top_laptops.to_json(orient='records')

Now that you have the `compare_laptops_with_user()` function ready, let's pass the `response_dict_n` to the function to get top 3 recommendation.

In [ ]:
top_3_laptops = compare_laptops_with_user(response_dict_n)
print(top_3_laptops)

[{"Brand":"Lenovo","Model Name":"ThinkPad","Core":"Ryzen 7","CPU Manufacturer":"AMD","Clock Speed":"3.0 GHz","RAM Size":"16GB","Storage Type":"SSD","Display Type":"IPS","Display Size":"14\"","Graphics Processor":"NVIDIA GTX","Screen Resolution":"2560x1440","OS":"Linux","Laptop Weight":"1.6 kg","Special Features":"Backlit Keyboard","Warranty":"3 years","Average Battery Life":"6 hours","Price":60000,"Description":"The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing good processing capabilities for demanding tasks. With 16GB of RAM and an SSD,  it offers good multitasking performance along with ample storage capacity. The laptop features a 14\" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It also comes with an NVIDIA GTX graphics card for enhanced graphical performance. Weighing just 1.6 kg, it offers decent portability. The laptop features a ba

Next, there is a function that verifies if the recommendations are good enough for the user's requirement.

In [ ]:
def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            data1.append(data[i])

    return data1

In [ ]:
validated_data = recommendation_validation(top_3_laptops)
print(validated_data)

[{'Brand': 'Lenovo', 'Model Name': 'ThinkPad', 'Core': 'Ryzen 7', 'CPU Manufacturer': 'AMD', 'Clock Speed': '3.0 GHz', 'RAM Size': '16GB', 'Storage Type': 'SSD', 'Display Type': 'IPS', 'Display Size': '14"', 'Graphics Processor': 'NVIDIA GTX', 'Screen Resolution': '2560x1440', 'OS': 'Linux', 'Laptop Weight': '1.6 kg', 'Special Features': 'Backlit Keyboard', 'Warranty': '3 years', 'Average Battery Life': '6 hours', 'Price': 60000, 'Description': 'The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing good processing capabilities for demanding tasks. With 16GB of RAM and an SSD,  it offers good multitasking performance along with ample storage capacity. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It also comes with an NVIDIA GTX graphics card for enhanced graphical performance. Weighing just 1.6 kg, it offers decent port

Now that you the top 3 laptops extracted, let's pass it to the recommendation layer that'll send it to the user and the user can ask questions around it.

In [ ]:
len(validated_data)

3

### Product Recommendation Layer

Finally, we come to the product recommendation layer. It takes the output from the `compare_laptops_with_user` function in the previous layer and provides the recommendations to the user. It has the following steps.
1. Initialize the conversation for recommendation.
2. Generate the recommendations and display in a presentable format.
3. Ask questions basis the recommendations.



In [ ]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue: {products}.\
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

Let's initialize the conversation for recommendation.

In [ ]:
debug_conversation_reco = initialize_conv_reco(top_3_laptops)

Let's see what the assistant responds with the new initialization.

In [ ]:
debug_recommendation = get_chat_model_completions(debug_conversation_reco)
print(debug_recommendation + '\n')

1. Acer Predator: Intel Core i7 processor, 16GB RAM, SSD storage, 17.3" IPS display, NVIDIA GTX graphics card, dual cooling fans, 1-year warranty, 5-hour battery life. Price: Rs 80,000.

2. Lenovo ThinkPad: AMD Ryzen 7 processor, 16GB RAM, SSD storage, 14" IPS display, NVIDIA GTX graphics card, backlit keyboard, 3-year warranty, 6-hour battery life. Price: Rs 60,000.

3. MSI GL65: Intel Core i7 processor, 16GB RAM, HDD+SSD storage, 15.6" IPS display, NVIDIA GTX graphics card, RGB keyboard, 2-year warranty, 4-hour battery life. Price: Rs 55,000.



Now, you can converse with the chatbot on the filtered products.

In [ ]:
debug_conversation_reco.append({"role": "user", "content": "This is my user profile" + response_dict_n})
debug_conversation_reco.append({"role": "assistant", "content": debug_recommendation})

In [ ]:
debug_user_input = "Which is ideal for travel?"

In [ ]:
debug_conversation_reco.append({"role": "user", "content": debug_user_input})
debug_response_asst_reco = get_chat_model_completions(debug_conversation_reco)
print('\n' + debug_response_asst_reco + '\n')


Based on your user profile, where portability is rated as low, none of the laptops mentioned in the catalogue are specifically designed for travel. However, among the options, the Lenovo ThinkPad with a weight of 1.6 kg could be considered relatively more portable compared to the other laptops.



### Dialogue Management System

Bringing everything together, we create a `diagloue_mgmt_system()` function that contains the logic of how the different layers would interact with each other. This will be the function that we'll call to initiate the chatbot

In [ ]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)
    print(introduction + '\n')
    top_3_laptops = None
    user_input = ''

    while(user_input != "exit"):
        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_3_laptops is None:
            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_model_completions(conversation)

            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            confirmation = intent_confirmation_layer(response_assistant)

            moderation = moderation_check(confirmation)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            if "No" in confirmation:
                conversation.append({"role": "assistant", "content": response_assistant})
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
            else:
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
                response = dictionary_present(response_assistant)

                moderation = moderation_check(response)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                print('\n' + response + '\n')
                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_laptops = compare_laptops_with_user(response)

                validated_reco = recommendation_validation(top_3_laptops)

                if len(validated_reco) == 0:
                    print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
                    break

                conversation_reco = initialize_conv_reco(validated_reco)
                recommendation = get_chat_model_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "user", "content": "This is my user profile" + response})

                conversation_reco.append({"role": "assistant", "content": recommendation})

                print(recommendation + '\n')

        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_model_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

## Areas of Improvement and Final Comments

In [ ]:
gamer_conversation = [{"role":"assistant","content":"Hi there! I'm an intelligent laptop gadget expert and I'm here to help you find the best laptop for your needs. Please tell me a little bit about yourself and your requirements so that I can assist you better."},
{"role":"user","content":"Yes, I prefer gaming with my graphics at high setting"},
{"role":"assistant","content":"Thank you for letting me know. Based on your preference for high graphics settings, I would recommend a laptop with a high GPU intensity. This will ensure that you can enjoy your games with excellent visual quality. Now, I have a question to better understand your needs. Are you primarily looking for a laptop that you can easily carry around and game on the go, or do you mainly use it in a stationary location?"},
{"role":"user","content":"I prefer using my laptop at a stationary location"},
{"role":"assistant","content":"Thank you for providing that information. Since you primarily use your laptop in a stationary location, portability may not be a top priority for you. Now, I have one more question to help me understand your requirements better. Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."},
{"role":"user","content":"150000"}
]

gamer_conversation_initialize = initialize_conversation()
gamer_conversation_initialize.extend(gamer_conversation)
gamer_conversation_assistant_output = get_chat_model_completions(gamer_conversation_initialize)
gamer_response_dict = dictionary_present(gamer_conversation_assistant_output)
gamer_dict = extract_dictionary_from_string(gamer_response_dict)
print(gamer_dict)

{'gpu intensity': 'high', 'display quality': 'high', 'portability': 'low', 'multitasking': 'medium', 'processing speed': 'medium', 'budget': '150000'}


In [ ]:
gamer_tagged_dict = {'gpu intensity': 'high', 'display quality': 'high', 'portability': 'low', 'multitasking': 'high', 'processing speed': 'high', 'budget': '150000'}
print(gamer_tagged_dict)

{'gpu intensity': 'high', 'display quality': 'high', 'portability': 'low', 'multitasking': 'high', 'processing speed': 'high', 'budget': '150000'}


In [ ]:
academic_conversation = [{"role":"assistant","content":"Hi there! I'm an intelligent laptop gadget expert and I'm here to help you find the best laptop for your needs. Please tell me a little bit about yourself and your requirements so that I can assist you better."},
{"role":"user","content":"i am a student and i'm looking to buy some laptops"},
{"role":"assistant","content":"Great! As a student, you likely have specific requirements for a laptop that will support your academic needs. Could you please provide me with more information about your requirements? Are you looking for a laptop for general use, such as browsing the internet, taking notes, and writing assignments? Or do you have any specific software or tasks that you need the laptop to handle? Understanding your needs will help me recommend the best laptop for you."},
{"role":"user","content":"I am working as a academic researcher and I do work with softwares and programming languages such as Matlab, Python and CATIA"},
{"role":"assistant","content":"Thank you for providing that information. As an academic researcher, you require a laptop that can handle software and programming tasks efficiently. May I know if you work with large datasets or perform any intensive computations? Understanding the nature of your work will help me determine the processing power and multitasking capabilities needed for your laptop."},
{"role":"user","content":"Yes, occasionally. I'm currently also working on a project on computer vision"},
{"role":"assistant","content":"Thank you for sharing that. Working on computer vision projects can be resource-intensive, especially when dealing with large datasets and complex algorithms. It requires a laptop with high processing power and a capable graphics card. Do you also require a high-quality display for accurate visualization of images and videos? Additionally, do you prioritize portability, as you may need to work on your projects in different locations?"},
{"role":"user","content":"I need a decent display and I also carry my laptop to college and conferences"},
{"role":"assistant","content":"Understood. Based on your requirements as an academic researcher working with software and programming languages like Matlab, Python, and CATIA, as well as computer vision projects, I would recommend the following specifications for your laptop: - GPU intensity: High (for handling computer vision tasks) - Display quality: Medium (for decent display) - Portability: Medium (since you carry your laptop to college and conferences) - Multitasking: High (to handle multiple software and programming tasks) - Processing speed: High (for efficient computation) Now, I need to know your budget for the laptop. Could you please let me know your maximum budget in INR?"},
{"role":"user","content":"approximately 100000"}
]

academic_conversation_initialize = initialize_conversation()
academic_conversation_initialize.extend(academic_conversation)
academic_conversation_assistant_output = get_chat_model_completions(academic_conversation_initialize)
academic_response_dict = dictionary_present(academic_conversation_assistant_output)
academic_dict = extract_dictionary_from_string(academic_response_dict)
print(academic_dict)

{'gpu intensity': 'high', 'display quality': 'medium', 'portability': 'medium', 'multitasking': 'high', 'processing speed': 'high', 'budget': 100000}


In [ ]:
academic_tagged_dict = {'gpu intensity': 'high', 'display quality': 'medium', 'portability': 'medium', 'multitasking': 'medium', 'processing speed': 'high', 'budget': '100000'}
print(academic_tagged_dict)

{'gpu intensity': 'high', 'display quality': 'medium', 'portability': 'medium', 'multitasking': 'medium', 'processing speed': 'high', 'budget': '100000'}


In [ ]:
business_conversation = [
{"role":"assistant","content":"Hi there! I'm an intelligent laptop gadget expert and I'm here to help you find the best laptop for your needs. Please tell me a little bit about yourself and your requirements so that I can assist you better."},
{"role":"user","content":"Hi. I a a business executive. I'm in the market to buy a laptop that fits my needs"},
{"role":"assistant","content":"Great! As a business executive, having a reliable and efficient laptop is crucial for your work. May I know what kind of tasks you primarily focus on? Are you involved in tasks such as data analysis, presentations, or multitasking between various applications? Understanding your specific work requirements will help me recommend the best laptop for you. "},
{"role":"user","content":"I'll use my laptop for day-to-day work such as attending zoom calls with clients, preparting reports and doing market research. I also do analysis on Excel"},
{"role":"assistant","content":"Thank you for providing that information. Based on your work requirements, it seems like you need a laptop that can handle multitasking, has good processing speed, and a decent display quality for presentations and data analysis. Portability may not be a top priority since you mentioned that you primarily work from a stationary location. Do you have any specific preferences or requirements for the graphics performance of the laptop?"},
{"role":"user","content":"I require a highly portable machine as I'll be commuting to work and also travelling to client locations. Graphics is not a priority for me"},
{"role":"assistant","content":"Understood. In that case, a laptop with high portability and multitasking capabilities would be ideal for you. Since graphics performance is not a priority, we can focus more on processing speed and display quality. Do you have any specific budget in mind for your laptop purchase? Knowing your budget will help me find options that fit within your price range while meeting your specified requirements."},
{"role":"user","content":"No budget is not a constraint. choose the best"},
{"role":"assistant","content":"Thank you for letting me know. In that case, I will recommend a laptop with high processing speed, good display quality, and excellent portability. This will ensure that you can efficiently handle your day-to-day work tasks, attend Zoom calls, prepare reports, and perform market research without any performance issues. Here are the specifications for the laptop I recommend for you: - GPU intensity: Low - Display quality: High - Portability: High - Multitasking: High - Processing speed: High - Budget: Not specified Please note that the budget is not specified as you mentioned that it is not a constraint. However, if you have any specific budget in mind, please let me know and I can provide you with more tailored recommendations."},
{"role":"user","content":"okay. maximum 200000"}
]

business_conversation_initialize = initialize_conversation()
business_conversation_initialize.extend(business_conversation)
business_conversation_assistant_output = get_chat_model_completions(business_conversation_initialize)
business_response_dict = dictionary_present(business_conversation_assistant_output)
business_dict = extract_dictionary_from_string(business_response_dict)
print(business_dict)

{'gpu intensity': 'low', 'display quality': 'high', 'portability': 'high', 'multitasking': 'high', 'processing speed': 'high', 'budget': '200000'}


In [ ]:
business_tagged_dict = {'gpu intensity': 'low', 'display quality': 'high', 'portability': 'high', 'multitasking': 'high', 'processing speed': 'high', 'budget': '200000'}
print(business_tagged_dict)

{'gpu intensity': 'low', 'display quality': 'high', 'portability': 'high', 'multitasking': 'high', 'processing speed': 'high', 'budget': '200000'}


In [ ]:
def evaluate_model_response(tagged_dict, model_dict):
  score = 0
  mappings = {
      'low': 0,
      'medium': 1,
      'high': 2
  }

  for key in tagged_dict.keys():
    if key == 'budget':
      continue
    tagged_value = tagged_dict[key]
    model_value = model_dict[key]
    tagged_mapping = mappings.get(tagged_value, -1)
    model_mapping = mappings.get(model_value, -1)

    if model_mapping >= tagged_mapping:
        score += 1

  return score


In [ ]:
gamer_score = evaluate_model_response(gamer_tagged_dict,gamer_dict)
print(gamer_score)
academic_score = evaluate_model_response(academic_tagged_dict,academic_dict)
print(academic_score)
business_score = evaluate_model_response(business_tagged_dict,business_dict)
print(business_score)

3
5
5


You can see that for gamer, the model is not able to perform well. But for the academic and business persona, it is able to.

### Future Scope of Work

1. The output format of each layer is inconsistent. You can use the function API capability of GPT to instruct the output format as per the input request.
2. The rule framework provided to classify each laptop’s specification is not exhaustive. You can expand the rules to give a comprehensive context to the LLM.
3. There are misclassifications in the laptop’s specifications, even after specifying clear rules for LLM. You can fine-tune an open-source LLM to make its understanding & performance better.
4. Once the products are extracted, the dialogue flow doesn’t allow recalling of product extraction if there is any intent change. You can add another layer to observe any request for a change in the user intent and then use this flag to recall the product extraction based on the updated intent.
5. As an alternative & simple solution, you can use vector embeddings of each product and compare it with the user intent to find the most relevant products.
6. You can template this workflow/solution to build a chatbot for any product domain. Note: You must add the relevant domain expertise/rules to give the LLM context understanding.
